In [2]:
import os
import sys

# Get the current working directory and add the parent directory to the Python path
current_working_directory = os.getcwd()
print(os.path.join(current_working_directory, ".."))
sys.path.append(os.path.join(current_working_directory, ".."))

/Users/L024258/lilly_work/github-copilot/exploration/notebooks/..


In [3]:
from decouple import AutoConfig
config = AutoConfig(search_path='./../.env')

In [4]:
os.environ["AZURE_OPENAI_API_KEY"] = config('OPENAI_API_KEY')
os.environ["AZURE_OPENAI_ENDPOINT"] = config('AZURE_ENDPOINT')

## Functions, Models, and Chains

### Function Binding

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from models.llm import LLM

In [ ]:
def joke_generator(topic):
    "This function will return a joke based on the topic provided"
    model = LLM('gpt-4')
    llm = model.load_model()
    prompt = ChatPromptTemplate.from_template(
        "Write a short joke on {topic}"
    )
    output_parser = StrOutputParser()
    chain = prompt | llm | output_parser
    return chain.invoke({"topic":topic})
    

In [ ]:
response = joke_generator("Artificial Intelligence")
print(response)

In [ ]:
def poem_generator(topic):
    "This function will return a poem based on the topic provided"
    model = LLM('gpt-4')
    llm = model.load_model()
    prompt = ChatPromptTemplate.from_template(
        "Write a short poem on {topic}"
    )
    output_parser = StrOutputParser()
    chain = prompt | llm | output_parser
    return chain.invoke({"topic":topic})

In [ ]:
response = poem_generator("Artificial Intelligence")
print(response)

#### OpenAI functon format

In [ ]:
functions =[
    {
      "name": "joke_generator",
      "description": "Generates a joke based on the topic provided",
      "parameters": {
        "type": "object",
        "properties": {
          "topic": {
            "type": "string",
            "description": "The topic to get the joke for"
          },
        },
        "required": ["topic"]
      }
    },
    {
      "name": "poem_generator",
      "description": "Generates a poem based on the topic provided",
      "parameters": {
        "type": "object",
        "properties": {
          "topic": {
            "type": "string",
            "description": "The topic to get the poem for"
          },
        },
        "required": ["topic"]
      }
    }
]

#### Attaching functions with model invocation

In [ ]:
messages = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a smart and intelligent AI Assistant."),
        ("user", "{input}")
    ]
)

In [ ]:
from models.llm import LLM

model = LLM('gpt-4')
gpt4 = model.load_model()

In [ ]:
gpt4.invoke(input="poem on Artificial Intelligence", functions=functions)

**NOTE:** With this approach, it is required to attach functions with everytime model is invoked.

#### Binding model with functions

In [ ]:
from langchain_openai import AzureChatOpenAI
gpt4 = AzureChatOpenAI(
                openai_api_version=config('AZURE_CHAT_OPENAI_API_VERSION'),
                azure_deployment=config('AZURE_GPT4_CHAT_OPENAI_DEPLOYMENT'),
                temperature=0,
                max_tokens=1024
            ).bind(functions=functions)

In [ ]:
runnable = messages | gpt4

In [ ]:
runnable.invoke({"input": "joke about Artificial Intelligence"})

In [ ]:
runnable.invoke({"input": "poem on Artificial Intelligence"})

## Tools and Models

In [4]:
from langchain_community.tools.arxiv.tool import ArxivQueryRun
from langchain_community.tools.pubmed.tool import PubmedQueryRun

arxiv_search = ArxivQueryRun()
pubmed_search = PubmedQueryRun()

tools = [arxiv_search, pubmed_search]

In [5]:
arxiv_search.name

'arxiv'

In [6]:
arxiv_search.description

'A wrapper around Arxiv.org Useful for when you need to answer questions about Physics, Mathematics, Computer Science, Quantitative Biology, Quantitative Finance, Statistics, Electrical Engineering, and Economics from scientific articles on arxiv.org. Input should be a search query.'

In [7]:
print(arxiv_search({"query": "LLM Agents"}))

/Users/L024258/lilly_work/github-copilot/exploration/.venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Published: 2024-05-23
Title: Exploring Prosocial Irrationality for LLM Agents: A Social Cognition View
Authors: Xuan Liu, Jie Zhang, Song Guo, Haoyang Shang, Chengxu Yang, Quanyan Zhu
Summary: Large language models (LLMs) have been shown to face hallucination issues due
to the data they trained on often containing human bias; whether this is
reflected in the decision-making process of LLM agents remains under-explored.
As LLM Agents are increasingly employed in intricate social environments, a
pressing and natural question emerges: Can LLM Agents leverage hallucinations
to mirror human cognitive biases, thus exhibiting irrational social
intelligence? In this paper, we probe the irrational behavior among
contemporary LLM agents by melding practical social science experiments with
theoretical insights. Specifically, We propose CogMir, an open-ended Multi-LLM
Agents framework that utilizes hallucination properties to assess and enhance
LLM Agents' social intelligence through cognitive bia

In [8]:
from langchain.tools.render import format_tool_to_openai_function
format_tool_to_openai_function(arxiv_search)

/Users/L024258/lilly_work/github-copilot/exploration/.venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The function `format_tool_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 0.3.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  warn_deprecated(


{'name': 'arxiv',
 'description': 'A wrapper around Arxiv.org Useful for when you need to answer questions about Physics, Mathematics, Computer Science, Quantitative Biology, Quantitative Finance, Statistics, Electrical Engineering, and Economics from scientific articles on arxiv.org. Input should be a search query.',
 'parameters': {'type': 'object',
  'properties': {'query': {'description': 'search query to look up',
    'type': 'string'}},
  'required': ['query']}}

In [9]:
from models.llm import LLM

model = LLM('gpt-4')
gpt4 = model.load_model()

In [10]:
functions = [
    format_tool_to_openai_function(f) for f in [
        arxiv_search, pubmed_search
    ]
]
model = gpt4.bind(function=functions)

In [11]:
tools = [arxiv_search, pubmed_search]
tools_model = gpt4.bind_tools(tools=tools)

In [13]:
tools_model.invoke("survey on LLM Agents")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_mdnjeXnlzttjUQyTNtvvBDfd', 'function': {'arguments': '{\n  "query": "survey on LLM Agents"\n}', 'name': 'arxiv'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 162, 'total_tokens': 181}, 'model_name': 'gpt-4-32k', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None, 'content_filter_results': {}}, id='run-d395f8a7-8177-4d1c-8040-66eeb114886e-0', tool_calls=[{'name': 'arxiv', 'args': {'query': 'survey on LLM Agents'}, 'id': 'call_mdnjeXnlzttjUQyTNtvvBDfd', 'type': 'tool_call'}], usage_metadata={'input_tokens': 162, 'output_tokens': 19, 'total_tokens': 181})

In [14]:
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [17]:
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful assistant"),
    ("user", "{input}"),
])
chain = prompt | tools_model | OpenAIFunctionsAgentOutputParser()

In [20]:
response = chain.invoke({"input": "survey on LLM Agents"})

In [21]:
response

AgentFinish(return_values={'output': ''}, log='')